# Debugging Session: ParameterValidation Timeout

## 1. Current Status
- **Scenario**: `ParameterValidation`
- **Symptom**: `ReadTimeout` (HTTP 500 fixed, but now timing out).
- **Environment**: Windows, `anolis-runtime` (Consumer) <-> `anolis-provider-sim` (Provider) via Stdio Pipes.
- **Recent Changes**: 
    - Added `wait_for_data` (PeekNamedPipe) to `FramedStdioClient`.
    - Added Mutex to `ProviderHandle`.
    - Fixed error mapping in `handlers.cpp`.

## 2. Hypothesis
The `ReadTimeout` suggests the runtime is waiting for a response that never arrives or is not read correctly.
Potential causes:
1. **Partial Read Blocking**: `read_exact` calls `ReadFile` which might block indefinitely if `PeekNamedPipe` says data is available (e.g., 1 byte) but `read_exact` tries to read more (e.g., 4 bytes header).
2. **Provider Hang**: The provider might be stuck processing a request or writing a response.
3. **Pipe Buffer Issues**: `ReadFile` on Windows might not return exactly the requested number of bytes even if they are available, or might block if they aren't.

## 3. Investigation Plan
1. Review `framed_stdio_client.cpp` implementation of `read_exact` and `wait_for_data`.
2. Inspect if `read_exact` handles short reads correctly (it seems to loop, but `ReadFile` blocking behavior is key).
3. If `PeekNamedPipe` returns > 0, `ReadFile` should return immediately with *available* data, not necessarily *all* requested data.
4. However, if we loop in `read_exact` logic until `n` bytes are read, and the provider only sent a partial frame (unlikely for localhost but possible), we might block on the *subsequent* `ReadFile` call inside the loop.

## 4. Code Review: `framed_stdio_client.cpp`
```cpp
bool FramedStdioClient::wait_for_data(int timeout_ms) {
    // ...
    if (!PeekNamedPipe(stdout_read_, NULL, 0, NULL, &bytes_available, NULL)) { ... }
    if (bytes_available > 0) return true; // Returns true even if 1 byte available
    // ...
}

bool FramedStdioClient::read_exact(uint8_t *buf, size_t n) {
    size_t total = 0;
    while (total < n) {
        // ...
        if (!ReadFile(stdout_read_, buf + total, static_cast<DWORD>(n - total), &read, NULL)) { ... }
        // ...
        total += read;
    }
    return true;
}
```
**Problem**: `wait_for_data` returns true if *any* data is available. `read_exact` then enters a loop calling `ReadFile`. The first `ReadFile` will return what's available. If that is less than `n` (e.g. we need 4 bytes for header, but only got 1), the loop continues. The *next* `ReadFile` will block until more data arrives. If the provider crashed or is slow, we block indefinitely here, ignoring the outer timeout loop in `ProviderHandle`.

**Fix Strategy**: `read_exact` needs to be non-blocking or aware of the timeout. Since generic `read` doesn't support timeout on pipes easily without `select`/`poll` (or `WaitForSingleObject` on Overlapped I/O, which we aren't using), we should rely on `wait_for_data` *inside* the `read_exact` loop or ensure `wait_for_data` guarantees enough bytes for the *next immediate* operation (header).

## 5. Experiment
Modify `FramedStdioClient` to assume that if `wait_for_data` returns true, we can try to read. But if we are in `read_exact` and need more data, we should probably check `wait_for_data` again or enforce a timeout there too.

Actually, a simpler robust fix for `read_frame`:
1. Wait for 4 bytes (header size) specifically? No, `PeekNamedPipe` tells us how many.
2. We can change `read_exact` to take a timeout or use `PeekNamedPipe` to wait for data before *every* chunk read if we are worried about blocking.

Let's try to verify if `ParameterValidation` is indeed failing due to this by modifying `framed_stdio_client.cpp` to use a loop with `PeekNamedPipe` inside `read_exact`.


In [ ]:
pass

Starting runtime: d:\repos_feast\anolis\build\core\Release\anolis-runtime.exe
Sending Test 5 (Valid)...
Call duration: 0.045s
Response: 200
Terminating...
